In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
!pip install bayesian-optimization

In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, Flatten
import tensorflow as tf
import numpy as np
import os
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder

from bayes_opt import BayesianOptimization

In [4]:
train=pd.read_csv("./train_data.csv", encoding='cp949')
test=pd.read_csv("./test_data.csv", encoding='cp949')

## 전처리

In [5]:
#결측치 확인
def shownull(df):
    global missval_name
    print(f"데이터셋 차원 확인 \n {df.shape}")
    missval = df.isnull().sum()[df.isnull().sum()!=0]
    print(f"결측값 있는 변수 확인 \n {missval}")
    missval_name = missval.index # 결측치가 존재하는 열이 저장된 변수

#이상치 확인
def zscore_out(df, threshold = 3):
    mean = np.mean(df)
    std = np.std(df)
    z_score = np.abs((df - df.mean()) / df.std())
    outliers = np.where(z_score > threshold)
    # outlier_idx = outliers.nonzero()[0]
    print(outliers[0])

#시드고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(84)

### 결측치 대체

In [9]:
# Train set에 대해서 시간 변수 결측치 대체

train['regdate'] = pd.to_datetime(train['regdate'])

# 시작일과 종료일 설정
start_date = train['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = train['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(train)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(train.loc[i, 'regdate']):
        train.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

#제대로 작동됨

In [10]:
# Test set에 대해서 시간 변수 결측치 대체

test['regdate'] = pd.to_datetime(test['regdate'])

# 시작일과 종료일 설정
start_date = test['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = test['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(test)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(test.loc[i, 'regdate']):
        test.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

### 독립변수 추가

In [11]:
# TRAIN 요일 변수 추가
daysofweek = []
for i in range(len(train)) :
    dayweek = train.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

train['dayofweek'] = daysofweek



#월요일 0 ~ 일요일6

<!--  -->

In [12]:
# TEST 요일 변수 추가

daysofweek = []
for i in range(len(test)) :
    dayweek = test.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

test['dayofweek'] = daysofweek

In [13]:
# TRAIN 시간 -> 10진법 변환
train['time'] = train['regdate'].dt.strftime('%H:%M').str.split().str[0]
train['time'] = train['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [14]:
# TEST 시간 -> 10진법 변환
test['time'] = test['regdate'].dt.strftime('%H:%M').str.split().str[0]
test['time'] = test['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [15]:
it_is_not_na = np.where(~train[['PIR']].isnull())[0]

train.loc[it_is_not_na,['PIR']] = train.loc[it_is_not_na,['PIR']].astype(np.int64)

### 결측치 대체

In [16]:
def make_area(df) :
    # 입력된 날짜 변수의 차이 계산
    date_range = df['regdate'].max() - df['regdate'].min()

    # 간격당 행의 개수 계산
    lowsbydate = ((date_range.days + 1) * 1440) #하루 1440분
    group_num = len(train) / lowsbydate

    for i in range(int(group_num)):
        start_index = i * lowsbydate
        end_index = (i + 1) * lowsbydate
        df.loc[start_index : end_index, 'area'] = i + 1

def inter_by_area(df):
    df.update(df.groupby('area').transform(lambda x: x.interpolate(method = 'pad')))
    return df

In [ ]:
train_nona = train.copy()
# cat_num_na(train_nona)
make_area(train_nona)
inter_by_area(train_nona)

test_nona = test.copy()
# cat_num_na(test_nona)
make_area(test_nona)
inter_by_area(test_nona)

### 파일 분할 및 스케일링

In [21]:
X_train = train_nona.drop('재실인원', axis=1)
X_train = X_train.drop(['regdate','area','dayofweek'], axis=1)
X_train= X_train.values
y_train = train_nona['재실인원'].values


X_test = test_nona.drop('재실인원', axis=1)
X_test = X_test.drop(['regdate','area','dayofweek'], axis=1)
X_test=X_test.values
y_test = test_nona['재실인원'].values

In [22]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

## timestep 지정을 위한 X값 shape 조정

In [23]:
timestep = 1
X_train_scaled = []
y_train_scaled = []
for i in range(len(train_nona) - timestep + 1):
    X_train_scaled.append(X_train[i:i+timestep])
    y_train_scaled.append(y_train[i+timestep-1])
X_train = np.array(X_train_scaled)
y_train = np.array(y_train_scaled)

In [24]:
X_test_scaled = []
y_test_scaled = []
for i in range(len(test_nona) - timestep + 1):
    X_test_scaled.append(X_test[i:i+timestep])
    y_test_scaled.append(y_test[i+timestep-1])
X_test = np.array(X_test_scaled)
y_test = np.array(y_test_scaled)

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [26]:
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(311040, 1, 11) (77760, 1, 11) (311040,) (77760,)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,LSTM
from keras.layers import Input, Dense, Activation, Flatten, Dropout

In [28]:
mse =[]
def buildModel(neurons,drop,nb_batch,hidden_layers):  
    nb_epoch=5
    neurons=int(neurons)
    hidden_layers=int(hidden_layers)

    # input layer
    inputs = tf.keras.Input(shape=(timestep,11))
    x = tf.keras.layers.LSTM(neurons*(2**(hidden_layers+1)), return_sequences=True, recurrent_dropout=drop, activation='relu')(inputs)  
    if hidden_layers !=0:
        for i in range(1,hidden_layers+1):
            x = tf.keras.layers.LSTM(neurons * (2 ** (hidden_layers + 1)), return_sequences=True, recurrent_dropout=drop)(x)
    # output
    predictions = Dense(1)(x)


    # modeling
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode="min",patience=3,restore_best_weights=True, verbose=1)



    # learning
    model.fit(X_train, y_train, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping], validation_data=(X_valid, y_valid))


    # evaluate the model with test data
    loss, mse =model.evaluate(X_test, y_test)
    
    if mse < 3.0 : 
        model.save(f"./model/lstm/{mse}_lstm.h5")
        print("모델이 저장 되었습니다.")
        print("mse : ",mse)
    
    return mse

In [29]:
pbounds = {
        # define the range of each parameter
        'drop' : (0.5,0.9),
        "neurons" : (12,24),
        'nb_batch' : (512,1024),
        'hidden_layers' : (0,3)}

In [30]:
bo=BayesianOptimization(f=buildModel, pbounds=pbounds, verbose=2, random_state=42 )

In [31]:
bo.maximize(
   init_points=2,
   n_iter= 100
)

|   iter    |  target   |   drop    | hidden... | nb_batch  |  neurons  |
-------------------------------------------------------------------------
Epoch 1/5
352/352 [==============================] - 22s 50ms/step - loss: 2.1876 - mean_squared_error: 2.1876 - val_loss: 2.0598 - val_mean_squared_error: 2.0598
Epoch 2/5
352/352 [==============================] - 19s 53ms/step - loss: 2.0743 - mean_squared_error: 2.0743 - val_loss: 2.0579 - val_mean_squared_error: 2.0579
Epoch 3/5
352/352 [==============================] - 20s 58ms/step - loss: 2.0739 - mean_squared_error: 2.0739 - val_loss: 2.0560 - val_mean_squared_error: 2.0560
Epoch 4/5
352/352 [==============================] - 20s 56ms/step - loss: 2.0741 - mean_squared_error: 2.0741 - val_loss: 2.0563 - val_mean_squared_error: 2.0563
Epoch 5/5
2657/2657 [==============================] - 20s 8ms/step - loss: 5.5697 - mean_squared_error: 5.5697
| 1         | 5.57      | 0.6498    | 2.852     | 886.8     | 19.18     |
Epoch 1/5
575/

Epoch 1/5
338/338 [==============================] - 11s 28ms/step - loss: 2.4300 - mean_squared_error: 2.4300 - val_loss: 2.1036 - val_mean_squared_error: 2.1036
Epoch 2/5
338/338 [==============================] - 9s 27ms/step - loss: 2.1023 - mean_squared_error: 2.1023 - val_loss: 2.0660 - val_mean_squared_error: 2.0660
Epoch 3/5
338/338 [==============================] - 9s 27ms/step - loss: 2.0775 - mean_squared_error: 2.0775 - val_loss: 2.0589 - val_mean_squared_error: 2.0589
Epoch 4/5
338/338 [==============================] - 9s 26ms/step - loss: 2.0740 - mean_squared_error: 2.0740 - val_loss: 2.0564 - val_mean_squared_error: 2.0564
Epoch 5/5
2657/2657 [==============================] - 12s 5ms/step - loss: 70515.6797 - mean_squared_error: 70515.6797
| 7         | 7.052e+04 | 0.6886    | 0.4815    | 922.6     | 13.16     |
Epoch 1/5
420/420 [==============================] - 31s 60ms/step - loss: 2.1549 - mean_squared_error: 2.1549 - val_loss: 2.0703 - val_mean_squared_error: 2

Epoch 5/5
2657/2657 [==============================] - 13s 5ms/step - loss: 54485.0625 - mean_squared_error: 54485.0625
| 18        | 5.449e+04 | 0.5918    | 0.7924    | 866.7     | 20.11     |
Epoch 1/5
583/583 [==============================] - 39s 58ms/step - loss: 2.1462 - mean_squared_error: 2.1462 - val_loss: 2.0564 - val_mean_squared_error: 2.0564
Epoch 2/5
583/583 [==============================] - 33s 57ms/step - loss: 2.0748 - mean_squared_error: 2.0748 - val_loss: 2.0559 - val_mean_squared_error: 2.0559
Epoch 3/5
583/583 [==============================] - 33s 57ms/step - loss: 2.0748 - mean_squared_error: 2.0748 - val_loss: 2.0559 - val_mean_squared_error: 2.0559
Epoch 4/5
583/583 [==============================] - 33s 57ms/step - loss: 2.0747 - mean_squared_error: 2.0747 - val_loss: 2.0574 - val_mean_squared_error: 2.0574
Epoch 5/5
2657/2657 [==============================] - 21s 8ms/step - loss: 2.4282 - mean_squared_error: 2.4282
모델이 저장 되었습니다.
mse :  2.428178071975708
| 1

Epoch 1/5
368/368 [==============================] - 25s 57ms/step - loss: 2.1696 - mean_squared_error: 2.1696 - val_loss: 2.0564 - val_mean_squared_error: 2.0564
Epoch 2/5
368/368 [==============================] - 20s 55ms/step - loss: 2.0745 - mean_squared_error: 2.0745 - val_loss: 2.0584 - val_mean_squared_error: 2.0584
Epoch 3/5
368/368 [==============================] - 20s 55ms/step - loss: 2.0747 - mean_squared_error: 2.0747 - val_loss: 2.0564 - val_mean_squared_error: 2.0564
Epoch 4/5
368/368 [==============================] - 20s 55ms/step - loss: 2.0747 - mean_squared_error: 2.0747 - val_loss: 2.0559 - val_mean_squared_error: 2.0559
Epoch 5/5
2657/2657 [==============================] - 20s 7ms/step - loss: 2.7835 - mean_squared_error: 2.7835
모델이 저장 되었습니다.
mse :  2.783540964126587
| 30        | 2.784     | 0.5372    | 2.254     | 848.0     | 22.13     |
Epoch 1/5
413/413 [==============================] - 11s 24ms/step - loss: 2.3908 - mean_squared_error: 2.3908 - val_loss: 

458/458 [==============================] - 29s 54ms/step - loss: 2.1698 - mean_squared_error: 2.1698 - val_loss: 2.0580 - val_mean_squared_error: 2.0580
Epoch 2/5
458/458 [==============================] - 24s 52ms/step - loss: 2.0750 - mean_squared_error: 2.0750 - val_loss: 2.0561 - val_mean_squared_error: 2.0561
Epoch 3/5
458/458 [==============================] - 24s 53ms/step - loss: 2.0739 - mean_squared_error: 2.0739 - val_loss: 2.0609 - val_mean_squared_error: 2.0609
Epoch 4/5
458/458 [==============================] - 24s 53ms/step - loss: 2.0743 - mean_squared_error: 2.0743 - val_loss: 2.0574 - val_mean_squared_error: 2.0574
Epoch 5/5
458/458 [==============================] - 24s 52ms/step - loss: 2.0748 - mean_squared_error: 2.0748 - val_loss: 2.0585 - val_mean_squared_error: 2.0585
Epoch 5: early stopping
2657/2657 [==============================] - 19s 7ms/step - loss: 6.5784 - mean_squared_error: 6.5784
| 42        | 6.578     | 0.5897    | 2.422     | 680.6     | 14.57  

...

Epoch 1/5
356/356 [==============================] - 10s 24ms/step - loss: 2.4905 - mean_squared_error: 2.4905 - val_loss: 2.1081 - val_mean_squared_error: 2.1081
Epoch 2/5
356/356 [==============================] - 9s 24ms/step - loss: 2.1040 - mean_squared_error: 2.1040 - val_loss: 2.0666 - val_mean_squared_error: 2.0666
Epoch 3/5
356/356 [==============================] - 8s 24ms/step - loss: 2.0771 - mean_squared_error: 2.0771 - val_loss: 2.0573 - val_mean_squared_error: 2.0573
Epoch 4/5
356/356 [==============================] - 8s 24ms/step - loss: 2.0738 - mean_squared_error: 2.0738 - val_loss: 2.0568 - val_mean_squared_error: 2.0568
Epoch 5/5
2657/2657 [==============================] - 11s 4ms/step - loss: 60785.3086 - mean_squared_error: 60785.3086
| 65        | 6.079e+04 | 0.8085    | 0.647     | 874.8     | 12.05     |
Epoch 1/5
327/327 [==============================] - 23s 56ms/step - loss: 2.2032 - mean_squared_error: 2.2032 - val_loss: 2.0596 - val_mean_squared_error: 2

2657/2657 [==============================] - 18s 7ms/step - loss: 2.8269 - mean_squared_error: 2.8269
모델이 저장 되었습니다.
mse :  2.8268518447875977
| 70        | 2.827     | 0.6038    | 2.387     | 721.7     | 12.59     |
Epoch 1/5
580/580 [==============================] - 37s 57ms/step - loss: 2.1326 - mean_squared_error: 2.1326 - val_loss: 2.0604 - val_mean_squared_error: 2.0604
Epoch 2/5
580/580 [==============================] - 32s 56ms/step - loss: 2.0746 - mean_squared_error: 2.0746 - val_loss: 2.0563 - val_mean_squared_error: 2.0563
Epoch 3/5
580/580 [==============================] - 33s 56ms/step - loss: 2.0748 - mean_squared_error: 2.0748 - val_loss: 2.0599 - val_mean_squared_error: 2.0599
Epoch 4/5
580/580 [==============================] - 32s 55ms/step - loss: 2.0747 - mean_squared_error: 2.0747 - val_loss: 2.0560 - val_mean_squared_error: 2.0560
Epoch 5/5
2657/2657 [==============================] - 18s 7ms/step - loss: 2.3545 - mean_squared_error: 2.3545
모델이 저장 되었습니다.
mse : 

Epoch 4/5
365/365 [==============================] - 15s 42ms/step - loss: 2.0740 - mean_squared_error: 2.0740 - val_loss: 2.0579 - val_mean_squared_error: 2.0579
Epoch 5/5
2657/2657 [==============================] - 15s 6ms/step - loss: 2.9109 - mean_squared_error: 2.9109
모델이 저장 되었습니다.
mse :  2.910872459411621
| 76        | 2.911     | 0.7755    | 1.811     | 853.9     | 18.99     |
Epoch 1/5
522/522 [==============================] - 14s 23ms/step - loss: 2.3587 - mean_squared_error: 2.3587 - val_loss: 2.0864 - val_mean_squared_error: 2.0864
Epoch 2/5
522/522 [==============================] - 12s 23ms/step - loss: 2.0851 - mean_squared_error: 2.0851 - val_loss: 2.0578 - val_mean_squared_error: 2.0578
Epoch 3/5
522/522 [==============================] - 12s 23ms/step - loss: 2.0741 - mean_squared_error: 2.0741 - val_loss: 2.0562 - val_mean_squared_error: 2.0562
Epoch 4/5
522/522 [==============================] - 12s 22ms/step - loss: 2.0735 - mean_squared_error: 2.0735 - val_loss: 

Epoch 4/5
313/313 [==============================] - 8s 26ms/step - loss: 2.0739 - mean_squared_error: 2.0739 - val_loss: 2.0562 - val_mean_squared_error: 2.0562
Epoch 5/5
2657/2657 [==============================] - 12s 4ms/step - loss: 44330.1328 - mean_squared_error: 44330.1328
| 88        | 4.433e+04 | 0.6136    | 0.147     | 995.8     | 16.67     |
Epoch 1/5
311/311 [==============================] - 17s 45ms/step - loss: 2.3459 - mean_squared_error: 2.3459 - val_loss: 2.0622 - val_mean_squared_error: 2.0622
Epoch 2/5
311/311 [==============================] - 13s 43ms/step - loss: 2.0748 - mean_squared_error: 2.0748 - val_loss: 2.0561 - val_mean_squared_error: 2.0561
Epoch 3/5
311/311 [==============================] - 13s 42ms/step - loss: 2.0734 - mean_squared_error: 2.0734 - val_loss: 2.0559 - val_mean_squared_error: 2.0559
Epoch 4/5
311/311 [==============================] - 13s 42ms/step - loss: 2.0735 - mean_squared_error: 2.0735 - val_loss: 2.0604 - val_mean_squared_error:

2657/2657 [==============================] - 16s 6ms/step - loss: 16.5583 - mean_squared_error: 16.5583
| 99        | 16.56     | 0.615     | 1.602     | 912.2     | 15.39     |
Epoch 1/5
307/307 [==============================] - 15s 39ms/step - loss: 2.3101 - mean_squared_error: 2.3101 - val_loss: 2.0596 - val_mean_squared_error: 2.0596
Epoch 2/5
307/307 [==============================] - 11s 37ms/step - loss: 2.0742 - mean_squared_error: 2.0742 - val_loss: 2.0566 - val_mean_squared_error: 2.0566
Epoch 3/5
307/307 [==============================] - 12s 38ms/step - loss: 2.0735 - mean_squared_error: 2.0735 - val_loss: 2.0561 - val_mean_squared_error: 2.0561
Epoch 4/5
307/307 [==============================] - 11s 37ms/step - loss: 2.0737 - mean_squared_error: 2.0737 - val_loss: 2.0567 - val_mean_squared_error: 2.0567
Epoch 5/5
2657/2657 [==============================] - 15s 6ms/step - loss: 5.8677 - mean_squared_error: 5.8677
| 100       | 5.868     | 0.798     | 1.565     | 1.015e+0

In [32]:
best_param=bo.res[60]['params']
best_param

{'drop': 0.7454776176361501,
 'hidden_layers': 1.7149540574533022,
 'nb_batch': 708.8028942286705,
 'neurons': 16.224301916835792}

In [33]:
reconstructed_model = keras.models.load_model("./model/lstm/1.6025696992874146_lstm.h5")

## MSE 값

In [34]:
reconstructed_model.evaluate(X_test,y_test)

2657/2657 [==============================] - 14s 5ms/step - loss: 1.6026 - mean_squared_error: 1.6026


[1.6025696992874146, 1.6025696992874146]

## RMSE 값

In [35]:
np.sqrt(reconstructed_model.evaluate(X_test, y_test))

2657/2657 [==============================] - 15s 6ms/step - loss: 1.6026 - mean_squared_error: 1.6026


array([1.26592642, 1.26592642])